# Database Admin 101 - Lab

## Introduction 

In this lab you'll go through the process of designing and creating a database. From there, you'll also begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Create a SQL database
* Create a SQL table
* Create rows in a SQL table
* Alter entries in a SQL table
* Delete entries in a SQL table
* Commit changes via sqlite3


## The Scenario

You are looking to design a database for a school which will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database and record some of your ideas below.

# Your Answer Here

Record some information here about how you would design such a database. What table would you include? What columns would each table have? What would be the primary means to join said tables?

* Table 1: grades(names\*, subject\*, grade)
* Table 2: contacts(name\*, address, email, phone)
* Table 3: Attendance(name\*, date*, attended)

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
#Your code here; import necessary packages
import sqlite3

In [2]:
#Your code here; create the database school.sqlite
conn= sqlite3.connect('school.sqlite')
c = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), email, telephone number, street, city, state and zipcode. Be sure to also create a primary key for the table. 

In [3]:
query = """
            CREATE TABLE contactInfo(
                                        id INTEGER PRIMARY KEY,
                                        firstName TEXT,
                                        lastName TEXT, 
                                        role TEXT,
                                        email TEXT,
                                        telephone TEXT,
                                        street TEXT,
                                        city TEXT,
                                        state TEXT,
                                        zipcode INTEGER
                                        )
                """
c.execute(query)

In [4]:
import pandas as pd
pd.read_sql("select * from contactInfo",conn)

,id,firstName,lastName,role,email,telephone,street,city,state,zipcode


In [7]:
#turns out that email column is not needed so we drop it. 
#renaming column name as there were typo
#1. query1 :rename contactInfo to temp_contact
#2. query2 :create table contactInfo with column names without email
#3. query3:copy data from temp_contact to contactInfo
#4 query4: drop the temp table
query1 ="""
            ALTER TABLE contactInfo RENAME TO temp_contact;
            
            """
query2='''CREATE TABLE contactInfo (id INTEGER PRIMARY KEY,
                                        firstName TEXT,
                                        lastName TEXT, 
                                        role TEXT,
                                        telephone TEXT,
                                        street TEXT,
                                        city TEXT,
                                        state TEXT,
                                        zipcode INTEGER);
            '''
query3="""INSERT INTO contactInfo(id, firstName,
                                        lastName, 
                                        role,
                                        telephone,
                                        street,
                                        city,
                                        state,
                                        zipcode)
                SELECT id, firstName,
                                        lastName, 
                                        role,
                                        telephone,
                                        street,
                                        city,
                                        state,
                                        zipcode
                FROM temp_contact;"""
query4="DROP TABLE temp_contact"

In [8]:
c.execute(query1)

In [9]:
c.execute(query2)

In [10]:
c.execute(query3)

In [11]:
c.execute(query4)

In [12]:
#checking if the table is droped
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('contactInfo',)]

In [13]:
pd.read_sql("select * from contactInfo",conn)

,id,firstName,lastName,role,telephone,street,city,state,zipcode


## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [14]:
# Code to load the list of dictionaries; just run this cell
import pickle
with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [15]:
print('test: {}'.format(tuple(contacts[0].values())))

test: ('Christine', 'Holden', 'staff', 2035687697, '1672 Whitman Court', 'Stamford', 'CT', '06995')


In [16]:
# Your code to iterate over the contact list and populate the contactInfo table here
# tuple(contacts[0].values())
for contact in contacts:
    c.execute("""
                    INSERT INTO contactInfo ( firstName,
                                                lastName, 
                                                role,
                                                telephone,
                                                street,
                                                city,
                                                state,
                                                zipcode)
                                        VALUES {}
                """.format(tuple(contact.values())))


In [17]:
# Query the Table to Ensure it is populated
pd.read_sql("select * from contactInfo",conn)

,id,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [25]:
#Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [18]:
#Your code here; create the grades table.
query="""CREATE TABLE grades(
                                userId INTEGER NOT NULL,
                                courseId INTEGER NOT NULL, 
                                grade INTEGER,
                                
                                PRIMARY KEY(userId, courseId)
                                
                                );

        """
c.execute(query)

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [19]:
#Your code here; find the duplicate entry
query="""SELECT firstName,lastName, telephone , COUNT(*) FROM contactInfo
            GROUP BY 1,2,3
            HAVING COUNT(*)>1;
            """
df=pd.read_sql(query, conn)

In [20]:
df

,firstName,lastName,telephone,COUNT(*)
0,Jane,Evans,3259909290,2


In [51]:
# c.execute('''DELETE FROM contactInfo WHERE telephone = 3259909290;''')

In [21]:
#Your code here; delete the duplicate entry
dups=[]
dups2=[]
for idx in range(len(df)):
    query = """SELECT id, telephone 
            FROM contactInfo 
            WHERE (firstName == "{}")&(lastName == "{}")&(telephone=="{}")""".format(df.iloc[idx][0],df.iloc[idx][1],df.iloc[idx][2])
    ids = pd.read_sql(query,conn)

    if len(ids)>1:
        for ID in range(len(ids)-1):
            dups.append(ids.iloc[ID].id)
            dups2.append((ids.iloc[ID].id, ids.iloc[ID].telephone))


In [22]:
dups2

[(5, '3259909290')]

In [23]:
for info in range(len(dups)):
    c.execute('''DELETE FROM contactInfo WHERE id ={};'''.format(dups[info]))

In [24]:
#Your code here; check that the duplicate entry was removed.
query="""SELECT firstName,lastName,telephone, COUNT(*) FROM contactInfo GROUP BY 1,2,3 HAVING COUNT(*)>1"""
pd.DataFrame(c.execute(query).fetchall(), columns = [x[0] for x in c.description])

,firstName,lastName,telephone,COUNT(*)


In [26]:
pd.read_sql("select * from contactInfo",conn)

,id,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Updating an Address

Ed Lyman just moved to 2910 Simpson Avenue York, PA 17403. Update his address accordingly.

In [28]:
#Your code here; update Ed's address
query = """
            UPDATE contactInfo
            SET street = '2910 Simpson Avenue',
                city = 'York',
                state = 'PA',
                zipcode = '17403'
                WHERE firstName = 'Ed' and lastName = 'Lyman';
            """
c.execute(query)

In [29]:
pd.read_sql("select * from contactInfo",conn)

,id,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,6995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [30]:
#Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you certainly got a taste for creating, populating and maintaining databases! Feel free to continue fleshing out this exercise to continue practicing. 